In [1]:
from __future__ import absolute_import, division, print_function, unicode_literals

# TensorFlow and tf.keras
import tensorflow as tf
from tensorflow import keras

# Helper libraries
import numpy as np
import matplotlib.pyplot as plt

from sklearn.linear_model import LogisticRegression
from sklearn.decomposition import NMF
import numpy.random as rand
from sklearn.metrics import roc_curve,auc
import numpy.linalg as la

from ml_enc import sNMF_L1
n_comp = 5

rand.seed(123)

### Load the data

In [2]:
X = np.genfromtxt('X_mod_fashion.csv',delimiter=',')
Y = np.genfromtxt('Y_fashion.txt')

### Determine the classes we wish to compare

In [3]:
class_1 = 2
class_2 = 4
indxs = (class_1==Y)|(class_2==Y)

X = X[indxs]
Y = Y[indxs]
Y[class_1==Y] = 0
Y[class_2==Y] = 1

In [4]:
idx_train = rand.binomial(1,size=X.shape[0],p=.7)
X_train = X[idx_train==1]
Y_train = Y[idx_train==1]
X_test = X[idx_train==0]
Y_test = Y[idx_train==0]

## Fit NMF Model

In [5]:
modNMF = NMF(n_comp)
modNMF.fit(X_train)

NMF(alpha=0.0, beta_loss='frobenius', init=None, l1_ratio=0.0, max_iter=200,
    n_components=5, random_state=None, shuffle=False, solver='cd', tol=0.0001,
    verbose=0)

In [6]:
S_train = modNMF.transform(X_train)
S_test = modNMF.transform(X_test)

## Fit logistic regression

In [7]:
model_lr = LogisticRegression()
model_lr.fit(S_train,Y_train)

/home/austin/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='warn', n_jobs=None, penalty='l2',
                   random_state=None, solver='warn', tol=0.0001, verbose=0,
                   warm_start=False)

## Evaluate the predictive performance

In [8]:
y_pred = model_lr.decision_function(S_test)
fpr,tpr,_ = roc_curve(Y_test,y_pred)
print(auc(fpr,tpr))

0.7585365169778849
